# CAI Jokes

In [16]:
import os
import openai
from dotenv import load_dotenv
import concurrent.futures
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

number_prompts = 5

constitution = """

1. Jokes should be sad

""".strip()


In [12]:
def get_concepts_prompt(n: int):
    return f"Generate a list of {n} forms of transport. Write each one on a separate, numbered line."


def get_naive_response_prompt(concept: str):
    return f"Write a joke about the following mode of transport:\n\n{concept}"


def get_critique(constitution: str, naive_response: str):
    return f"""Here is a joke:
<joke>
{naive_response}
</joke>

And here are some rules about the jokes I want to tell:

<rules>
{constitution}
</rules>

Which rules does this joke break? Please explain your reasoning either way."""


## Generate concepts

In [4]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
          "role": "user",
          "content": get_concepts_prompt(number_prompts)
        }
    ],
    max_tokens=1024,
    temperature=0.7,
    stream=False
)

concepts_string = completion.choices[0].message.content
print(concepts_string)


1. Car
2. Train
3. Bicycle
4. Boat
5. Airplane


In [5]:
def getItems(str):
    items = []
    for line in str.splitlines():
        items.append(line[line.find(".") + 2:])
    return items


concepts = getItems(concepts_string)
if len(concepts) != number_prompts:
    raise Exception(f"Expected {number_prompts} concepts, but got {len(concepts)}")
else:
    print(f"Got {len(concepts)} concepts")


Got 5 concepts


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [6]:
def get_llm_response(concept, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": get_naive_response_prompt(concept)
            }
        ],
        max_tokens=1024,
        temperature=0.3,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str


with concurrent.futures.ThreadPoolExecutor() as executor:
    naive_responses = list(executor.map(lambda x: get_llm_response(
        x[0], x[1]), [(concept, i) for i, concept in enumerate(concepts)]))

tasksResponsesString = "\n\n---\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
print(tasksResponsesString)


Response 2 received
Response 0 received
Response 4 received
Response 1 received
Response 3 received
TASK:
Car

NAIVE RESPONSE:
Why did the car break up with the bicycle? Because it wanted to see other vehicles.

---

TASK:
Train

NAIVE RESPONSE:
Why did the train break up with his girlfriend? Because she was always locomotivated!

---

TASK:
Bicycle

NAIVE RESPONSE:
Why did the bicycle fall over? Because it was two-tired!

---

TASK:
Boat

NAIVE RESPONSE:
Why did the sailor bring a ladder on the boat? 

Because he wanted to climb aboard!

---

TASK:
Airplane

NAIVE RESPONSE:
Why did the airplane break up with his girlfriend? Because he was always flying solo!


Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique naive responses

In [17]:
def get_llm_critique(constitution, naive_response, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": get_critique(constitution, naive_response)
            }
        ],
        max_tokens=1024,
        temperature=0.5,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str


with concurrent.futures.ThreadPoolExecutor() as executor:
    critiques = list(executor.map(lambda x: get_llm_critique(
        constitution, x[0], x[1]), [(response, i) for i, response in enumerate(naive_responses)]))

responseCritiqueString = "\n\n---\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


Response 2 received
Response 0 received
Response 4 received
Response 1 received
Response 3 received
NAIVE RESPONSE:
Why did the car break up with the bicycle? Because it wanted to see other vehicles.

CRITIQUE:
This joke breaks the rule that jokes should be sad, as it is a lighthearted play on words and not meant to evoke sadness.

---

NAIVE RESPONSE:
Why did the train break up with his girlfriend? Because she was always locomotivated!

CRITIQUE:
This joke breaks the rule that jokes should be sad, as it is a pun-based joke that is meant to be humorous.

---

NAIVE RESPONSE:
Why did the bicycle fall over? Because it was two-tired!

CRITIQUE:
This joke breaks the rule that jokes should be sad, as it is a pun and meant to be humorous.

---

NAIVE RESPONSE:
Why did the sailor bring a ladder on the boat? 

Because he wanted to climb aboard!

CRITIQUE:
This joke breaks the given rule that jokes should be sad. It is a pun-based joke that relies on a play on words to create humor, which is ty

In [11]:
print(get_critique(constitution, naive_responses[0]))


Here is a joke:
<joke>
Why did the car break up with the bicycle? Because it wanted to see other vehicles.
</joke>

And here are some rules about the jokes I want to tell:

<rules>
1. Jokes should express irony.
</rules>

Which rules does this joke break? If it doesn't break any rules, please respond with "None".
